# Segmenting and clustering neighborhood in Toronto

In this notebook, we are gonna first retrieve the different neighborhood of Toronto on Wikipedia

## Scraping Wikipedia to get the neighborhoods

In [1]:
! easy_install beautifulsoup4
! pip install lxml

print('beautifulsoup4 is now installed !')

Searching for beautifulsoup4
Best match: beautifulsoup4 4.7.1
Adding beautifulsoup4 4.7.1 to easy-install.pth file

Using /home/jupyterlab/conda/lib/python3.6/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4
beautifulsoup4 is now installed !


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


 
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "lxml") 

In [3]:
table=soup.find('table')
table=table.text
table[:200]

'\n\nPostcode\nBorough\nNeighbourhood\n\n\nM1A\nNot assigned\nNot assigned\n\n\nM2A\nNot assigned\nNot assigned\n\n\nM3A\nNorth York\nParkwoods\n\n\nM4A\nNorth York\nVictoria Village\n\n\nM5A\nDowntown Toronto\nHarbourfront\n\n\nM5A\n'

In [4]:
rows=table.split('\n\n\n')
for i in range(len(rows)):
    rows[i]=rows[i].split('\n')
rows[0].pop(0)
rows[0].pop(0)
rows[-1].pop(-1)
rows[-1].pop(-1)

rows

df=pd.DataFrame(columns=rows[0],data=rows[1:])
df=df[df['Borough']!='Not assigned']
df.head()


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
def neighbourhood_list(grouped):
    if grouped['Neighbourhood'].tolist()[0]=='Not assigned':
        return(grouped['Borough'].tolist()[0])
    # transform grouped Neighborhood as single value separated with commas
    return ', '.join(sorted(grouped['Neighbourhood'].tolist())) 
                    
grp = df.groupby(['Postcode', 'Borough'])
df1=grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we are gonna <b>clean the data</b>. i.e. we are going to replace the <i>Not assigned</i> of the neighbourhood and join the different neighbourhood of a same postcode.

# Complete the data with geocoder

Because Google's geo API is not free anymore, I use arcgis.

In [6]:
! pip install geocoder
import geocoder

In [7]:
def get_latlng(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M1B')

[43.811525000000074, -79.19551721399995]

In [8]:
postal_codes = df1['Postcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]


In [9]:
df2=pd.DataFrame(data=coords, columns=['Latitude','Longitude'])

In [10]:
df1[['Latitude','Longitude']]=df2

In [11]:
df1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [13]:
df1.shape

(103, 5)

# Plotting the neighborhood on a map

First import all the libraries

In [14]:
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors


In [15]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
next(df1.iterrows())[1]['Borough']
toronto_center = pd.DataFrame([row for index, row in df1.iterrows() if 'Toronto' in row['Borough']])
toronto_center.reset_index(drop=True, inplace=True)
toronto_center.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_center['Latitude'],
                                            toronto_center['Longitude'],
                                            toronto_center['Borough'],
                                            toronto_center['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [47]:
# @hidden_cell
CLIENT_ID = 'HE5K3R53FXEMA5L2CYF1HVVICBSSV0ITHWZPVXLCCURU0CJV' # your Foursquare ID
CLIENT_SECRET = 'KYCMYLB1G204YJLOVKTEK0FR4SSSAXH0OFDCHEDDHOK1AERU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
toronto_center.loc[0, 'Neighbourhood']

'The Beaches'

In [21]:
neighborhood_latitude = toronto_center.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_center.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_center.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67684518300007, -79.29522499999996.


In [22]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HE5K3R53FXEMA5L2CYF1HVVICBSSV0ITHWZPVXLCCURU0CJV&client_secret=KYCMYLB1G204YJLOVKTEK0FR4SSSAXH0OFDCHEDDHOK1AERU&v=20180605&ll=43.67684518300007,-79.29522499999996&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc24017531593002c866167'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.68134518750007,
    'lng': -79.28901466400717},
   'sw': {'lat': 43.672345178500066, 'lng': -79.30143533599275}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Dip 'n Sip,Coffee Shop,43.678897,-79.297745
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_center['Neighbourhood'],
                                   latitudes=toronto_center['Latitude'],
                                   longitudes=toronto_center['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1744, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676845,-79.295225,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676845,-79.295225,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
4,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood


In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",65,65,65,65,65,65
Berczy Park,63,63,63,63,63,63
"Brockton, Exhibition Place, Parkdale Village",70,70,70,70,70,70
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",92,92,92,92,92,92
Christie,10,10,10,10,10,10


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


we now encode the different venues.

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
toronto_onehot.shape

(1744, 214)

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.010000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.015385,0.000000,0.000000,0.015385,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.015873,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.015873,0.000000,0.000000,0.000000,0.000000
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.042857,0.00,0.014286,...,0.000000,0.000000,0.00000,0.00,0.00,0.028571,0.000000,0.014286,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.010000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.024390,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.010000,0.01,0.000000,...,0.000000,0.000000,0.01000,0.00,0.00,0.010000,0.010000,0.010000,0.010000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.032609,0.00,0.010870,...,0.000000,0.010870,0.01087,0.00,0.00,0.054348,0.000000,0.043478,0.010870,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.011765,0.011765,0.011765,0.00,0.000000,0.000000,0.00,0.011765,...,0.011765,0.000000,0.00000,0.00,0.00,0.000000,0.011765,0.011765,0.000000,0.011765


In [35]:
toronto_grouped.shape

(38, 214)

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.05
3  Burger Joint  0.03
4    Steakhouse  0.03


----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.08
2                Café  0.05
3          Restaurant  0.05
4                 Pub  0.05


----Berczy Park----
          venue  freq
0   Coffee Shop  0.11
1  Cocktail Bar  0.05
2    Restaurant  0.05
3        Bakery  0.03
4        Lounge  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.06
2              Restaurant  0.06
3  Furniture / Home Store  0.06
4          Sandwich Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2   Restaurant  0.04
3    

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Bakery,Asian Restaurant,Restaurant,Bar,Steakhouse,Burger Joint,Deli / Bodega
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Coffee Shop,Italian Restaurant,Pub,Restaurant,Café,Gym / Fitness Center,Sandwich Place,Park,Diner,Speakeasy
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Breakfast Spot,Lounge,Steakhouse,Farmers Market,Bakery,Pub,Cheese Shop
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Sandwich Place,Art Gallery,Bar,Italian Restaurant,Supermarket,Bakery
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Bar,Restaurant,Steakhouse,Hotel,American Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant


### We apply the KMeans Method to cluster the neighbourhood

In [39]:
from sklearn.cluster import KMeans

In [40]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 4, 5, 5, 0, 5,
       0, 5, 3, 5, 0, 5, 0, 5, 2, 0, 1, 5, 5, 5, 5, 5], dtype=int32)

We add the labels in the Dataframe

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_center

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,5,Coffee Shop,Health Food Store,Pub,Wings Joint,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120,2,Bus Line,Park,Grocery Store,Discount Store,Wings Joint,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.667965,-79.314673,5,Sandwich Place,Pet Store,Italian Restaurant,Movie Theater,Pub,Fast Food Restaurant,Fish & Chips Shop,Burrito Place,Burger Joint,Park
3,M4M,East Toronto,Studio District,43.662766,-79.334830,5,Italian Restaurant,Diner,Bakery,Gastropub,Brewery,Bar,American Restaurant,Sushi Restaurant,Café,Coffee Shop
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,3,Bus Line,Swim School,Wings Joint,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Let's print the final map with the clustered neighbourhood

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters